# FLUX LoRA Inference

Author: [Zain Hasan](https://x.com/ZainHasan6)

---
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/togethercomputer/together-cookbook/blob/main/Agents/Flux_LoRA_Inference.ipynb)

## Introduction

In this notebook I'll show how to generate images by performing inference with LoRA fine-tunes of the FLUX.1-dev model.

We discuss the role of trigger words for different LoRA fine-tunes and how to think about mixing multiple LoRA tunes and scaling each accordingly.

<img src="images/lora_pandas.png" height="500">

Here is a list of interesting LoRA fine-tunes of the open source FLUX.1-dev model that you can try:

| Style | Link |
|-------|------|
| Colored Sketch | https://huggingface.co/strangerzonehf/Flux-Sketch-Ep-LoRA |
| Simple Sketch | https://huggingface.co/dvyio/flux-lora-simple-illustration |
| Vector Sketch | https://huggingface.co/mujibanget/vector-illustration |
| Anime Sketch | https://huggingface.co/glif/anime-blockprint-style |
| Logo Design | https://huggingface.co/Shakker-Labs/FLUX.1-dev-LoRA-Logo-Design |
| Flux Midjourney | https://huggingface.co/strangerzonehf/Flux-Midjourney-Mix2-LoRA |
| Outfit Generator | https://huggingface.co/tryonlabs/FLUX.1-dev-LoRA-Outfit-Generator |
| Icons | https://huggingface.co/strangerzonehf/Flux-Icon-Kit-LoRA |
| Tarot Card | https://huggingface.co/multimodalart/flux-tarot-v1 |
| Add Detail | https://huggingface.co/Shakker-Labs/FLUX.1-dev-LoRA-add-details |


In [ ]:
!pip install -qU together

In [ ]:
from together import Together
from IPython.display import Image
import os

client = Together(api_key=os.getenv("TOGETHER_API_KEY"))

In [42]:
from typing import Optional, List
from dataclasses import dataclass

@dataclass
class LoraConfig:
    path: str
    scale: float

def generate_image(
        prompt: str,
        lora1: str,
        scale1: float = 1.0,
        lora2: Optional[str] = None, 
        scale2: Optional[float] = 1.0
) -> str:
    
    """
    Generate an image using the FLUX.1 model with specified LoRA's.

    Args:
        prompt (str): The text prompt to generate the image from.
        lora1 (str): Path to the first LoRA model.
        scale1 (float, optional): Scale factor for the first LoRA model. Defaults to 1.0.
        lora2 (Optional[str], optional): Path to the second LoRA model. Defaults to None.
        scale2 (Optional[float], optional): Scale factor for the second LoRA model. Defaults to 1.0.

        Currently only 2 LoRA's at a time are supported.

    Returns:
        str: URL of the generated image.
    """
    
    # Build LoRA configurations
    lora_config: List[LoraConfig] = [LoraConfig(path=lora1, scale=scale1)]
    
    if lora2:
        lora_config.append(LoraConfig(path=lora2, scale=scale2))

    # Convert LoraConfig objects to dictionaries
    image_loras = [{"path": lora.path, "scale": lora.scale} for lora in lora_config]

    response = client.images.generate(
        prompt=prompt,
        model="black-forest-labs/FLUX.1-dev-lora",
        width=1024,
        height=768,
        steps=28,
        n=1,
        response_format="url",
        image_loras=image_loras
    )

    return response.data[0].url

In [ ]:
prompt = "baby panda eating bamboo"

generated_image = generate_image(prompt, 
                                 lora1="https://huggingface.co/multimodalart/flux-tarot-v1",
                                 scale1=1.0
                                 )

Image(url=generated_image, width=512, height=384)

### Trigger words to activate LoRA

The above image does not look like a tarot card! This is because many LoRA fine-tunes have trigger words that need to be included in the prompt in order for the fine-tune to work!

The trigger phrase for the tarot card LoRA fine-tune is: `in the style of TOK a trtcrd tarot style`.

You can typically find these trigger words with the model card for the LoRA fine-tune.

In [48]:
prompt = "a baby panda eating bamboo in the style of TOK a trtcrd tarot style"

generated_image = generate_image(prompt, 
                                 lora1="https://huggingface.co/multimodalart/flux-tarot-v1",
                                 scale1=1.0
                                 )

Image(url=generated_image, width=512, height=384)

### Using Multiple LoRA's

We can add another LoRA fine-tune from above into the mix to make the Tarot realistic.

The `scale` parameter allows you to specify the strength of each LoRA. Typically values of `0.3-1.2` will produce good results.

In [49]:
prompt = "a baby panda eating bamboo in the style of TOK a trtcrd tarot style"

generated_image = generate_image(prompt, 
                                 lora1="https://huggingface.co/multimodalart/flux-tarot-v1",
                                 scale1=1.0,
                                 lora2="https://huggingface.co/Shakker-Labs/FLUX.1-dev-LoRA-add-details",
                                 scale2=0.8
                                 )

Image(url=generated_image, width=512, height=384)

This produces a much more realistic looking panda still in the style of a tarot card!

### Negative Scales

Some LoRA fine-tunes also support negative scale, the realism LoRA we are using can take negative scales which if used will reduce the realism of the generated images.

In [58]:
prompt = "a baby panda eating bamboo in the style of TOK a trtcrd tarot style"

generated_image = generate_image(prompt, 
                                 lora1="https://huggingface.co/multimodalart/flux-tarot-v1",
                                 scale1=1.0,
                                 lora2="https://huggingface.co/Shakker-Labs/FLUX.1-dev-LoRA-add-details",
                                 scale2=-0.8
                                 )

Image(url=generated_image, width=512, height=384)

### Making Realistic Sketches

We can combine the sketch LoRA with the realism LoRA to generate a realistic sketches!

Note: the simple sketch LoRA has a trigger phrase: ` in the style of SMPL`

In [59]:
prompt = "an baby panda eating bamboo illustration in the style of SMPL"

generated_image = generate_image(prompt, 
                                lora1="https://huggingface.co/Shakker-Labs/FLUX.1-dev-LoRA-add-details",
                                scale1=0.9,
                                lora2="https://huggingface.co/dvyio/flux-lora-simple-illustration",
                                scale2=1.25
                                )

Image(url=generated_image, width=512, height=384)

To learn more about Flux LoRA Inference read our docs [here](https://docs.together.ai/docs/quickstart-flux-lora).